# Web Scraping Drudgereport
- This notebook is a tutorial on how to scrape the Drudgereport website with Python. 

In [1]:
# import all modules that will be used in our scraper

from requests import get 
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint 
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# request the contents of the page we're scraping

results = get('https://www.drudgereport.com/')

In [3]:
# make the content we grabbed easy to read

html_soup = BeautifulSoup(results.text, 'html.parser')

In [4]:
html_soup

 <html>
<head>
<meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy"/>
<title>DRUDGE REPORT 2022®</title>
<!-- eProof, DR Headline Updater -->
<script src="//cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript"></script>
<script src="//eproof.drudgereport.com/dr.js" type="text/javascript"></script>
<!-- eProof.com end -->
<script src="/js/cookie_filter_1.0.1.js" type="text/javascript"></script>
<script type="text/javascript">
<!-- 
var img = new Image(), url = "/204.png", container = document.getElementById("div-204");
img.onload = function () { container.appendChild(img); };
img.src = url;
var timer = setInterval("__drudge__321__autoRefresh()", 1000 * 35 * 3);
function __drudge__321__autoRefresh(){self.location.reload(true);}
(function () {
    var __oldClearInterval = clearInterval;

    clearInterval = function (arg) {
        if(arg == timer) {
                console.log("clearInterval of TIMER intercepted! (" + arg + ")");
   

In [5]:
# get the macro-container containing the posts we want

# post = html_soup.find_all('li', class_= 'result-row')

post = html_soup.find_all('a')

print(type(post)) # check I got a ResultSet
print(len(post)) # check I got 120 

<class 'bs4.element.ResultSet'>
236


In [6]:
# grab the first post

post_one = post[0]

In [7]:
print(post_one)

<a href="https://www.cnbc.com/2022/01/08/cyprus-reportedly-discovers-a-covid-variant-that-combines-omicron-and-delta.html" nopush="">'Deltacron' discovered in Cyprus...</a>


In [8]:
post

[<a href="https://www.cnbc.com/2022/01/08/cyprus-reportedly-discovers-a-covid-variant-that-combines-omicron-and-delta.html" nopush="">'Deltacron' discovered in Cyprus...</a>,
 <a href="https://news.yahoo.com/fed-pandemic-angelenos-time-learn-130018175.html" nopush="">ENOUGH: FED UP LEARN TO LIVE WITH IT...</a>,
 <a href="https://www.the-sun.com/health/4425231/covid-spreading-hotspots-sage-list/">Where you are most likely to catch?</a>,
 <a href="https://unherd.com/2022/01/how-vaccine-mandates-became-a-political-weapon/">Evidence against mandates piles up...</a>,
 <a href="https://www.msn.com/en-us/health/medical/florida-e2-80-99s-covid-cases-and-hospitalizations-are-surging-but-the-state-has-the-second-lowest-death-rate-in-the-nation-what-e2-80-99s-going-on/ar-AASyQKx">FL cases, hospitalizations surging, but has 2nd-lowest death rate. What's going on?</a>,
 <a href="https://www.arcamax.com/currentnews/newsheadlines/s-2617194">MODEL:  Wave peak mid-Jan...</a>,
 <a href="https://www.cnn.

In [12]:
#grab the text of the first post

post_one_text = post_one.text

In [13]:
post_one_text

"'Deltacron' discovered in Cyprus..."

In [15]:
# grab the link

post_one_link = post_one['href']
post_one_link

'https://www.cnbc.com/2022/01/08/cyprus-reportedly-discovers-a-covid-variant-that-combines-omicron-and-delta.html'

Goals:
Story = Story or Photo
Story arrives
Story departs
Story location changes
Story text changes
Story Size or Color changes
Story Duration

Constant Link arrives
Constant Link departs

Locations:
Top left
Top Center
Bottom Left
Bottom Center
Bottom Right

In [16]:
for link in html_soup.find_all('a'):
    print(link.get('href'))


https://www.cnbc.com/2022/01/08/cyprus-reportedly-discovers-a-covid-variant-that-combines-omicron-and-delta.html
https://news.yahoo.com/fed-pandemic-angelenos-time-learn-130018175.html
https://www.the-sun.com/health/4425231/covid-spreading-hotspots-sage-list/
https://unherd.com/2022/01/how-vaccine-mandates-became-a-political-weapon/
https://www.msn.com/en-us/health/medical/florida-e2-80-99s-covid-cases-and-hospitalizations-are-surging-but-the-state-has-the-second-lowest-death-rate-in-the-nation-what-e2-80-99s-going-on/ar-AASyQKx
https://www.arcamax.com/currentnews/newsheadlines/s-2617194
https://www.cnn.com/2022/01/09/politics/national-guard-hospitals-omicron-covid-massachusetts/index.html
https://apnews.com/article/coronavirus-pandemic-health-michigan-muskegon-0c3cda9fa32c62bf05c062e056be65ac
https://www.the-sun.com/news/4420761/meat-egg-shortages-fears-omicron/
https://news.yahoo.com/djokovic-opens-dramatic-court-battle-222827975.html
https://www.tmz.com/2022/01/09/bob-saget-dead-die

# Build the Loop

In [53]:
# find the total number of posts to find the limit of the pagination
find_total = html_soup.find('div', class_= 'search-legend')



# grab the total count of posts 
total_posts = int(find_total.find('span', class_='totalcount').text) 



# vary the value of the page parameters
pages = np.arange(0, total_posts+1, 120)



# vary the value of the page parameters
pages = np.arange(0, total_posts+1, 120)



# count tracker for number of iterations
iterations = 0



# initialize empty lists where we'll store our date 
post_times = []
post_neighborhoods = []
post_titles = []
post_bedrooms = []
post_sqft = []
post_links = []
post_prices = []



# create for loop
for page in pages:
    
    # get request
    response = get("https://boston.craigslist.org/search/apa?" 
                   + "s=" # parameter for defining the page number 
                   + str(page) # page number in the pages array 
                   + "&hasPic=1"
                   + "&availabilityMode=0")
    
    # control the crawl rate 
    sleep(randint(1,10))
    
    # throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    # define the html text
    html = BeautifulSoup(response.text, 'html.parser')
    
    # define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
    
    
    # extract data item-wise
    for post in posts:

        # if we aren't missing the neighborhood information
        if post.find('span', class_ = 'result-hood') is not None:

            # date
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_times.append(post_datetime)

            # neighborhoods
            post_hoods = post.find('span', class_= 'result-hood').text
            post_neighborhoods.append(post_hoods)

            # title 
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)

            # link
            post_link = post_title['href']
            post_links.append(post_link)
            
            # removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = int(float(post.a.text.strip().replace("$", "").replace(",","")))
            post_prices.append(post_price)
            
            
            
            
            
            # if the number of bedrooms OR sqft aren't missing 
            if post.find('span', class_ = 'housing') is not None:
                
                # if the first element is accidentally square footage
                if 'ft' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    # make bedroom NaN
                    bedroom_count = np.nan
                    post_bedrooms.append(bedroom_count)
                    
                    # make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    post_sqft.append(sqft)
                    
                # if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    # therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    post_bedrooms.append(bedroom_count)
                    
                    # and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    post_sqft.append(sqft)
                    
                # if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    # therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    post_bedrooms.append(bedroom_count)
                    
                    # and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    post_sqft.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    post_bedrooms.append(bedroom_count)
                
                    sqft = np.nan
                    post_sqft.append(sqft)
                
            # if none of those conditions catch, make bedroom NaN 
            else:
                bedroom_count = np.nan
                post_bedrooms.append(bedroom_count)
                
                sqft = np.nan
                post_sqft.append(sqft)
                
    iterations += 1
      
    print("Page " + str(iterations) + " scraped successfully!")

    
print("\n")
print("Scrape complete!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!


Scrape complete!


In [55]:
phx_apts = pd.DataFrame({'posted': post_times,
                       'neighborhood': post_neighborhoods,
                       'post title': post_titles,
                       'number bedrooms': post_bedrooms,
                        'sqft': post_sqft,
                        'URL': post_links,
                       'price': post_prices})

print(phx_apts.info())
phx_apts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   posted           1664 non-null   object 
 1   neighborhood     1664 non-null   object 
 2   post title       1664 non-null   object 
 3   number bedrooms  1560 non-null   object 
 4   sqft             1352 non-null   float64
 5   URL              1664 non-null   object 
 6   price            1664 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 91.1+ KB
None


,posted,neighborhood,post title,number bedrooms,sqft,URL,price
0,2021-01-26 19:44,(PHOENIX),"Dog Park, 24 hour Fitness Center, Garages Avai...",2,865.0,https://phoenix.craigslist.org/cph/apa/d/phoen...,1251
1,2021-01-26 19:42,"(5740 North 59th Ave, Glendale, AZ, US)",Make Ranchwood your home TODAY! Apply online f...,1,540.0,https://phoenix.craigslist.org/wvl/apa/d/glend...,888
2,2021-01-26 19:41,"(5740 North 59th Ave, Glendale, AZ, US)",High speed internet included! Make Ranchwood y...,1,676.0,https://phoenix.craigslist.org/wvl/apa/d/glend...,881
3,2021-01-26 19:27,(near Scottsdale Rd / McDowell),Home in Scottsdale,3,NaN,https://phoenix.craigslist.org/evl/apa/d/scott...,1895
4,2021-01-26 19:13,(Phoenix),Luxury studio residence atop Palomar Hotel in ...,1,840.0,https://phoenix.craigslist.org/cph/apa/d/phoen...,800
...,...,...,...,...,...,...,...
1659,2021-01-26 17:55,"(2625 E. Camelback Rd., Phoenix, AZ)","Sit Back, Relax. Your New View Awaits.",2,1074.0,https://phoenix.craigslist.org/evl/apa/d/phoen...,2408
1660,2021-01-26 17:53,"(2300 W Pecos Rd Chandler, AZ)",Come see our beautiful community!!!,1,756.0,https://phoenix.craigslist.org/evl/apa/d/chand...,1352
1661,2021-01-26 17:53,(Phoenix),Two bedroom in Beautiful Ahwatukee,2,1205.0,https://phoenix.craigslist.org/evl/apa/d/phoen...,1590
1662,2021-01-26 17:52,(Mesa),Stay & Play! Fun Amenities! Call Now!,2,828.0,https://phoenix.craigslist.org/evl/apa/d/mesa-...,1185


In [61]:
# to move all the scraped data to a CSV file

phx_apts.to_csv('phx_apts.csv', index=False)